In [ ]:
import osmnx as ox
import geopandas as gpd
import pandas as pd
import folium
from shapely.geometry import Point
from shapely.ops import unary_union, polygonize

# 1. 여의도 도로 네트워크 데이터 가져오기
# 여의도 지역의 도로 네트워크를 'drive' 타입으로 불러옴
place_name = "Yeouido-dong, Seoul, South Korea"
graph = ox.graph_from_place(place_name, network_type="drive")

# 도로 네트워크에서 노드와 엣지 데이터를 추출
nodes, edges = ox.graph_to_gdfs(graph)

# 도로 네트워크를 기반으로 폴리곤(구역) 생성
lines = edges['geometry'].values
merged_lines = unary_union(lines)  # 도로를 하나로 병합하여 경계 생성
polygons = list(polygonize(merged_lines))  # 병합된 도로를 기준으로 폴리곤 생성

# 폴리곤을 GeoDataFrame으로 변환하여 다룰 수 있도록 함
polygons_gdf = gpd.GeoDataFrame(geometry=polygons, crs="EPSG:4326")

# 2. 인구 데이터 불러오기
# 2023년 10월 7일의 데이터를 불러오기 위한 템플릿과 파일 경로 설정
base_path = "/content/drive/MyDrive/여의도_불꽃축제_데이터/인구종합/"
file_template = "2024-10-05_{:02d}_인구종합.csv"  # 2023년 데이터 템플릿

# 시간별 데이터를 하나의 DataFrame으로 병합
all_data = pd.DataFrame()
for hour in range(24):  # 00시부터 23시까지 반복하여 파일 처리
    file_path = base_path + file_template.format(hour)
    try:
        df = pd.read_csv(file_path)
        all_data = pd.concat([all_data, df])  # 파일 합치기
    except FileNotFoundError:
        print(f"파일이 없습니다: {file_path}")
        continue

# 3. 각 격자별 인구수 총합 계산
# 인구수 기준으로 각 좌표를 그룹화하여 합산
aggregated_population = all_data.groupby(['X_COORD', 'Y_COORD'])['POPL_CNT'].sum().reset_index()

# GeoDataFrame으로 변환하여 좌표를 다룰 수 있게 함
geometry = [Point(xy) for xy in zip(aggregated_population.X_COORD, aggregated_population.Y_COORD)]
population_gdf = gpd.GeoDataFrame(aggregated_population, geometry=geometry, crs="EPSG:4326")

# 4. 각 폴리곤별 인구 합산
# 각 도로 구역(폴리곤)에 포함된 인구 수를 합산하여 'POPULATION' 컬럼에 저장
polygons_gdf["POPULATION"] = 0  # 'POPULATION' 컬럼 초기화
for idx, polygon in polygons_gdf.iterrows():
    # 폴리곤 내에 포함된 인구 데이터를 필터링하여 합산
    mask = population_gdf.within(polygon.geometry)
    total_population = population_gdf.loc[mask, "POPL_CNT"].sum()
    polygons_gdf.at[idx, "POPULATION"] = total_population  # 각 폴리곤에 해당 인구 수 저장

# 5. 안전도 분석을 위한 위험도 계산
# 안전도가 높은 지역은 초록색, 낮은 지역은 빨간색으로 표시 (위험도가 높을수록 빨강)
max_population = polygons_gdf["POPULATION"].max()
min_population = polygons_gdf["POPULATION"].min()

# 6. 범위에 따른 색상 설정 (위험도 색상: 초록, 주황, 빨강)
def get_safety_color(population, min_pop, max_pop):
    # 인구수에 따라 초록(안전) -> 빨강(위험)으로 변하는 색상 설정
    if population <= min_pop + (max_pop - min_pop) / 3:
        return "green"  # 안전한 지역
    elif population <= min_pop + 2 * (max_pop - min_pop) / 3:
        return "orange"  # 주의가 필요한 지역
    else:
        return "red"  # 위험한 지역

# 7. Folium 지도 시각화
# 여의도의 지도를 생성
m = folium.Map(location=[37.526260, 126.926979], zoom_start=15)

# 각 폴리곤을 지도에 시각화
for _, row in polygons_gdf.iterrows():
    if row.POPULATION > 0:
        color = get_safety_color(row["POPULATION"], min_population, max_population)
        folium.Polygon(
            locations=[(point[1], point[0]) for point in row.geometry.exterior.coords],
            color=None,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
        ).add_to(m)

# 컬러맵 추가 (위험도 범위 설정)
m.add_child(folium.ColorMap(['green', 'orange', 'red'], ['Low Risk', 'Medium Risk', 'High Risk']))

# 지도 저장
map_file_path = "/content/drive/MyDrive/여의도_안전도_분석_구역별_지도.html"
m.save(map_file_path)

# 지도 표시
m  # 지도를 표시

# 저장된 지도 파일 경로 출력
print(f"위험도 분석 지도 파일이 저장되었습니다: {map_file_path}")